# Analyser les ventes d'un site de e-commerce spécialisé dans les livres

L'entreprise pour qui vous travaillez souhaite analyser les ventes de son site et mieux connaitre ses clients, ainsi que le détail des ventes de ses produits

Vous pourrez télécharger l'ensemble des données [via ce lien](https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/parcours-data-analyst/dataset_P4.zip)  Les données que vous avez à disposition :   
* les ventes (appelées “Transactions”)  
* la liste des clients  
* la liste des produits.

## Nettoyage des données

Avant de pouvoir entrer dans le vif du sujet, il vous faudra faire un peu de nettoyage ! Par exemple, vous devrez faire des choix quant au traitement des valeurs manquantes et des valeurs aberrantes.

Pour vous guider, voici quelques choses à explorer :  
* l'évolution du Chiffre d'Affaire de manière mensuelle
* Tester les différentes jointures possibles entre les tables (pour rappel, il en existe [4 types](http://python-simple.com/python-pandas/concatenations-joins-dataframe.php ))
* Des valeurs abérrantes sont présentes dans la table *transaction*, identifiez-les et supprimez-les de votre étude.
* Regarder le nombre de transaction par client ou le nombre de CA par client

Vous aurez donc des choix à faire : imputation de valeurs manquantes, gestions de valeurs atypiques etc.

A noter : enregistrer vos données nettoyées dans un fichier csv, nous le réutiliserons ensuite pour étudier les corrélations

In [1]:
import pandas as pd

In [4]:
df_transaction = pd.read_csv('./dataset/transactions.csv')
df_transaction

,id_prod,date,session_id,client_id
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277
2,1_374,2021-09-23 15:13:46.938559,s_94290,c_4270
3,0_2186,2021-10-17 03:27:18.783634,s_105936,c_4597
4,0_1351,2021-07-17 20:34:25.800563,s_63642,c_1242
...,...,...,...,...
337011,1_671,2021-05-28 12:35:46.214839,s_40720,c_3454
337012,0_759,2021-06-19 00:19:23.917703,s_50568,c_6268
337013,0_1256,2021-03-16 17:31:59.442007,s_7219,c_4137
337014,2_227,2021-10-30 16:50:15.997750,s_112349,c_5


In [41]:
from os import listdir
from pathlib import Path
directory_path = './dataset/'
listdir_data = listdir(directory_path)
listdir_data

['customers.csv', 'products.csv', 'transactions.csv']

Recommençons.

In [47]:
df_customers = pd.read_csv(directory_path + listdir_data[0])
df_products = pd.read_csv(directory_path + listdir_data[1])
df_customers,df_products

(     client_id sex  birth
 0       c_4410   f   1967
 1       c_7839   f   1975
 2       c_1699   f   1984
 3       c_5961   f   1962
 4       c_5320   m   1943
 ...        ...  ..    ...
 8618    c_7920   m   1956
 8619    c_7403   f   1970
 8620    c_5119   m   1974
 8621    c_5643   f   1968
 8622      c_84   f   1982
 
 [8623 rows x 3 columns],
      id_prod   price  categ
 0     0_1421   19.99      0
 1     0_1368    5.13      0
 2      0_731   17.99      0
 3      1_587    4.99      1
 4     0_1507    3.99      0
 ...      ...     ...    ...
 3282    2_23  115.99      2
 3283   0_146   17.14      0
 3284   0_802   11.22      0
 3285   1_140   38.56      1
 3286  0_1920   25.16      0
 
 [3287 rows x 3 columns])

<generator object <genexpr> at 0x000001A41217F0C8>

ValueError: Invalid file path or buffer object type: <class 'list'>